In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/sandbox/lsst/lsst/GUI/20180525_002_flats/'
%cd $thedir

#%matplotlib inline

In [ ]:
files = glob.glob('ITL-3800C-002_flat_flat_900_20180525??????.fits')
#files = sort(glob.glob('E2V-CCD250-112-09_flat_flat_400_20190405??????.fits'))
name = files[0][24:27]
print files
print name


In [ ]:
xstart = 505
xstop = 542
ov_start = 512
ystart = 200
ystop = 1800
xaxis = linspace(xstart,xstop-1,xstop-xstart)

for file in [files[0]]:
    print file
    name = file[24:27]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    figure(figsize=(15,15))
    subplots_adjust(hspace = 0.5, wspace = 0.5)
    suptitle("Serial CTE by segment - ITL CCD 002\n", fontsize = 16)
    for segment in range(16):
        try:
            subplot(4,4,segment+1)
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            title("CTE - %s"%extname)
            data = array(hdulist[segment+1].data + 32768, dtype = int32)
            flat_overscan = mean(array(data[:,xstop-8:xstop]),axis = 1)
            cte_data = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            cte_std = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = median((transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start]\
            / (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start-1]) / ov_start

            scatter(xaxis, cte_data, color = 'blue')
            errorbar(xaxis, cte_data, yerr=cte_std)
            text(xstart + 8, 20000, "CTI = %.2E"%cti, fontsize = 12)
            yscale('log',nonposy='clip')
            ylim(.01, 100000)
            xticks([510,520,530,540])
            xlabel("X (First overscan = 512)")
            ylabel("Signal(ADU)")
            print "Segment %s done"%extname
        except:
            continue
    savefig("Serial_CTE_%s_16Apr19.pdf"%name)
    clf()

# This was just a check to see if I got the same results as before - almost identical